In [8]:
import copy
import tequila as tq
from tequila import gates
tq.show_available_simulators()

backend         | wfn        | sampling   | noise      | installed 
--------------------------------------------------------------------
qulacs_gpu      | False      | False      | False      | False     
qulacs          | False      | False      | False      | False     
qibo            | False      | False      | False      | False     
qiskit          | True       | True       | True       | True      
cirq            | True       | True       | True       | True      
pyquil          | True       | True       | True       | True      
symbolic        | True       | False      | False      | True      
qlm             | False      | False      | False      | False     


In [9]:
def xor(q1: int, q2: int, output: int):
    circuit = tq.QCircuit()
    circuit += gates.CNOT(q1, output)
    circuit += gates.CNOT(q2, output)
    return circuit

In [10]:
def sudoku_oracle(inputs: list[int], tmp: list[int], output: int):
    circuit = tq.QCircuit()

    circuit += xor(inputs[0], inputs[1], tmp[0])
    circuit += xor(inputs[1], inputs[3], tmp[1])
    circuit += xor(inputs[0], inputs[2], tmp[2])
    circuit += xor(inputs[2], inputs[3], tmp[3])

    circuit_t = copy.deepcopy(circuit)

    circuit += gates.X(output, tmp)

    circuit += circuit_t

    return circuit

In [11]:
def reflect_zero(inputs: list[int]):
    circuit = tq.QCircuit()

    circuit += gates.X(inputs)
    circuit += gates.Z(inputs[-1], inputs[:len(inputs) - 1])
    circuit += gates.X(inputs)

    return circuit

In [12]:
def reflect_initial(init_op: tq.QCircuit, inputs: list[int]):
    circuit = tq.QCircuit()

    circuit += init_op.dagger()
    circuit += reflect_zero(inputs)
    circuit += init_op

    return circuit

In [13]:
def grover_circuit():
    inputs = [i for i in range(4)]
    tmp = [i + 4 for i in range(4)]
    pkb = 8

    circuit = tq.QCircuit()
    prepare_initial_state = gates.H(inputs)

    circuit += prepare_initial_state
    circuit += gates.X(pkb)
    circuit += gates.H(pkb)

    for _ in range(2):
        circuit += sudoku_oracle(inputs, tmp, pkb)
        circuit += reflect_initial(prepare_initial_state, inputs)

    return circuit

In [14]:
measurements = tq.simulate(grover_circuit(), samples=512, read_out_qubits=[0,1,2,3])
print(measurements)

+1.0000|0111> +1.0000|0011> +1.0000|0001> +257.0000|0110> +2.0000|1011> +7.0000|1100> +3.0000|1110> +2.0000|1101> +5.0000|0100> +3.0000|0000> +1.0000|1000> +1.0000|0010> +226.0000|1001> +1.0000|1010> +1.0000|0101> 
